Initial loading sequence. Only need to initialize this one time

In [37]:
#importing relevant libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB

import torch

import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer

import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

from keras.models import load_model


loadedModel = load_model('/content/bertmodelRegLabels.h5')



Data testing. If the printed number is less than 0.5, they're depressed, and if not they're suicidal. 

In [39]:
test_data = {0: ["I tried to hang myself today. It didn't work. I don't know where to go from here or what to do. I guess I'll try again some other way."]}
test_text = pd.DataFrame(test_data, columns = [0])

## Want BERT instead of distilBERT? Uncomment the following line:
model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

tokenized = test_text[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=128)))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

np.array(padded).shape

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

features = last_hidden_states[0][:,0,:].numpy()


predictions = loadedModel.predict(features)
print(predictions)

[[0.7012777]]
